In [2]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [3]:
# Import data\WalkabilityIndex\Natl_WI_simplified.csv using pandas
df = pd.read_csv(r'data\WalkabilityIndex\Natl_WI_simplified.csv')

# Convert WKT geometries to actual geometries
df['geometry'] = df['geometry'].apply(wkt.loads)

# Create a GeoDataFrame with the initial CRS (assuming EPSG:4326)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Display the first row
gdf.head(n=1)

,geoid10,geoid20,statefp,countyfp,tractce,blkgrpce,csa,csa_name,cbsa,cbsa_name,...,d3b,d4a,d2a_ranked,d2b_ranked,d3b_ranked,d4a_ranked,natwalkind,shape_length,shape_area,geometry
0,481130078254,481130078254,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,"Dallas-Fort Worth-Arlington, TX",...,115.981747,362.1,6.0,14.0,15.0,17.0,14.0,3110.36082,297836.08309,"POLYGON ((-10769245.79938 3880757.19657, -1076..."


In [4]:
# Trying to figure out how unproductive land works in the data set-> Probably not a good measure?
# Calculate the percent of unproductive land by total acres
gdf['percent_unproductive'] = round(gdf['ac_unpr'] / gdf['ac_total'] * 100, 2)

# Create bins for the percent_unproductive values
bins = range(0, 100, 10)
gdf['bins'] = pd.cut(gdf['percent_unproductive'], bins=bins)

# Group by the bins and count the geoid20s in each bin
bin_counts = gdf.groupby('bins', observed=False)['geoid20'].count()

# Convert the Series to a DataFrame
bin_counts_df = bin_counts.reset_index()
bin_counts_df.columns = ['bin', 'geoid20_count']

# Calculate the total count of geoid20
total_count = bin_counts_df['geoid20_count'].sum()

# Calculate the percentage for each bin
bin_counts_df['percentage'] = (bin_counts_df['geoid20_count'] / total_count) * 100

# Print the DataFrame
bin_counts_df

,bin,geoid20_count,percentage
0,"(0, 10]",889,2.036515
1,"(10, 20]",1112,2.547362
2,"(20, 30]",1515,3.470552
3,"(30, 40]",2039,4.670928
4,"(40, 50]",2637,6.040822
5,"(50, 60]",3697,8.469063
6,"(60, 70]",5192,11.893799
7,"(70, 80]",8642,19.797036
8,"(80, 90]",17930,41.073924


In [5]:
# Count the geoid20 per cbsa_name
cbsa_counts = gdf.groupby('cbsa_name')['geoid20'].count()

# Convert the Series to a DataFrame
cbsa_counts_df = cbsa_counts.reset_index()
cbsa_counts_df.columns = ['cbsa_name', 'geoid20_count']

# Calculate the total count of geoid20
total_geoid20_count = cbsa_counts_df['geoid20_count'].sum()

# Calculate the percentage for each cbsa_name
cbsa_counts_df['percentage'] = round((cbsa_counts_df['geoid20_count'] / total_geoid20_count) * 100, 2)

# Sort the DataFrame by geoid20_count in descending order
cbsa_counts_df.sort_values('geoid20_count', ascending=False, inplace=True)

# Display the DataFrame
cbsa_counts_df.head(n=1)

,cbsa_name,geoid20_count,percentage
604,"New York-Newark-Jersey City, NY-NJ-PA",14376,7.06


In [6]:
# Trying to figure out if it makes sense to drop the cases where the CBSA name is null
# Count the geoid20 where cbsa_name is null
null_cbsa_count = gdf[gdf['cbsa_name'].isnull()]['geoid20'].count()
print(f'Number of geoid20 where cbsa_name is null: {null_cbsa_count}')

# Count the total number of rows in the DataFrame
total_rows = len(gdf)
print(f'Total number of rows in the DataFrame: {total_rows}')

# Calculate the percentage of rows where cbsa_name is null
null_cbsa_percentage = (null_cbsa_count / total_rows) * 100
print(f'Percentage of rows where cbsa_name is null: {null_cbsa_percentage:.2f}%')

Number of geoid20 where cbsa_name is null: 17094
Total number of rows in the DataFrame: 220739
Percentage of rows where cbsa_name is null: 7.74%


In [7]:
# Looking to understand the distribution of the ac_total column
gdf['ac_total'].describe()

count    2.207390e+05
mean     1.103501e+04
std      1.588983e+05
min      1.096136e+00
25%      1.231823e+02
50%      3.394929e+02
75%      2.325853e+03
max      4.055649e+07
Name: ac_total, dtype: float64

Large Variability: The data exhibits a wide range of area sizes, from just over 1 acre to over 40 million acres. The large standard deviation and high maximum value indicate that while most areas are relatively small to moderate in size, there are a few extremely large areas that are significantly larger than the rest.

Skewed Distribution: The mean being much higher than the median suggests a right-skewed distribution, where the presence of a few very large values is pulling the average up. This is further evidenced by the 75th percentile being well below the mean.

Outliers: The maximum value of over 40 million acres is an outlier, and it could significantly affect the mean and standard deviation. Depending on the analysis, it might be important to consider how this outlier influences overall results.

In [9]:
# Ensure 'cbsa_name' is of type string before filtering
gdf['cbsa_name'] = gdf['cbsa_name'].astype(str)

# Filter the gdf where cbsa_name contains 'Knoxville'
knoxville_gdf = gdf[gdf['cbsa_name'].str.contains('Knoxville', case=False)]
knoxville_gdf.head()

,geoid10,geoid20,statefp,countyfp,tractce,blkgrpce,csa,csa_name,cbsa,cbsa_name,...,d2a_ranked,d2b_ranked,d3b_ranked,d4a_ranked,natwalkind,shape_length,shape_area,geometry,percent_unproductive,bins
200137,470930031002,470930031002,47,93,3100,2,315.0,"Knoxville-Morristown-Sevierville, TN",28940.0,"Knoxville, TN",...,10.0,10.0,7.0,13.0,10.000000,6822.411982,2.172040e+06,"POLYGON ((-9339559.44907 4302240.20113, -93395...",100.0,NaN
200138,470930032001,470930032001,47,93,3200,1,315.0,"Knoxville-Morristown-Sevierville, TN",28940.0,"Knoxville, TN",...,16.0,16.0,9.0,13.0,12.666667,7923.088459,2.379561e+06,"POLYGON ((-9337152.83295 4300562.20486, -93371...",100.0,NaN
200139,470930032002,470930032002,47,93,3200,2,315.0,"Knoxville-Morristown-Sevierville, TN",28940.0,"Knoxville, TN",...,6.0,3.0,12.0,17.0,11.166667,5906.734471,1.277692e+06,"POLYGON ((-9337096.95062 4300518.3113, -933707...",100.0,NaN
200140,470930039021,470930039021,47,93,3902,1,315.0,"Knoxville-Morristown-Sevierville, TN",28940.0,"Knoxville, TN",...,20.0,14.0,9.0,14.0,13.333333,4986.660142,1.216165e+06,"POLYGON ((-9348763.90112 4301202.05306, -93487...",100.0,NaN
200141,470930057042,470930057042,47,93,5704,2,315.0,"Knoxville-Morristown-Sevierville, TN",28940.0,"Knoxville, TN",...,19.0,16.0,9.0,13.0,13.166667,8540.755571,3.396974e+06,"POLYGON ((-9360962.95877 4289372.19761, -93609...",100.0,NaN
